In [2]:
import numpy as np
import cv2
import glob
print("Header files loaded!")

Header files loaded!


In [4]:
# least square method
def least_squares_method(a_matrix, b_matrix):
    
    # compute A transpose
    a_matrix_transpose = np.transpose(a_matrix)
    
    # compute (A transpose)*(A)
    a_transpose_a = np.matmul(a_matrix_transpose, a_matrix)

    # compute inverse of a_transpose_a
    a_transpose_a_inv = np.linalg.inv(a_transpose_a)

    # compute a_transpose_a_inv*a_matrix_transpose and multiply with b to get the answer
    coeff_of_b = np.matmul(a_transpose_a_inv, a_matrix_transpose)
    answer = np.matmul(coeff_of_b, b_matrix)

    # return answer
    return answer

In [3]:
path_of_images = "/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/*"
files  = glob.glob(path_of_images)
files = sorted(files)
print(files)

['/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0001.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0002.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0003.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0004.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0005.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0006.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0007.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0008.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0009.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0010.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0011.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0012.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/0013.jpg', '/home/arpitdec5/Desktop/lucas_kanade_tracker/data/Bolt2/img/00

In [17]:
# get the grayscale image after converting it to lab colorspace
def get_grayscale_image(image):
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(1, 1))
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    lab = cv2.merge((l, a, clahe.apply(b)))
    lab = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    gray = cv2.cvtColor(lab, cv2.COLOR_BGR2GRAY)
    return gray

# update the grayscale image, that is, normalize the pixel values with template image
def update_grayscale_image(template_image, image):
    template_mean = np.mean(template_image)
    mean = np.mean(image)
    image = (image * (template_mean / mean)).astype(float)
    return image

# computes error between images
def compute_error_images(image_array1, image_array2):
    return (image_array1 - image_array2)

# computes the coordinates of the ROI of interest
def get_coordinates_array(x_range, y_range):
    coordinates_array = np.zeros(3, (x_range[1] - x_range[0] + 1) * (y_range[1] - y_range[0] + 1))
    count = 0
    for y in range(y_range[0], y_range[1] + 1):
        for x in range(x_range[0], x_range[1] + 1):
            coordinates_array[0, count] = x
            coordinates_array[1, count] = y
            coordinates_array[2, count] = 1
            count = count + 1
    return coordinates_array

# computes the new coordinates of the image wrt template image
def get_new_image_coordinates(template_gray_image_coordinates_array, p, x_range, y_range):
    
    # define the rectangle coordinates of the ROI of template image
    template_rectangle_coordinates = np.array([[x_range[0], x_range[0], x_range[1], x_range[1]], [y_range[0], y_range[1], y_range[1], y_range[0]], [1, 1, 1, 1]])

    # get the affine matrix to get the warped image
    affine_matrix = np.zeros((2, 3))
    count = 0
    for i in range(0, 3):
        for j in range(0, 2):
            affine_matrix[j, i] = p[count, 0]
            count = count + 1
    affine_matrix[0, 0] = affine_matrix[0, 0] + 1
    affine_matrix[1, 1] = affine_matrix[1, 1] + 1
    
    # get new rectange coordinates
    new_rectangle_coordinates = np.dot(affine_matrix, template_rectangle_coordinates)
    
    # get new coordinates
    new_gray_image_coordinates_array = np.dot(affine_matrix, template_gray_image_coordinates_array)
    new_gray_image_coordinates_array = new_gray_image_coordinates_array.astype(int)
    
    # return the two arrays
    return (new_gray_image_coordinates_array, new_rectangle_coordinates)
    
# computes the pixel array
def get_pixel_array(image, image_coordinates_array):
    
    # get the pixel values of the ROI of the image
    pixel_array = np.zeros(1, image_coordinates_array.shape[1])
    pixel_array[0, :] = image[image_coordinates_array[1, :], image_coordinates_array[0, :]]
    return pixel_array
    
# compute the error in the p matrix
def get_delta_p(error, steep_descent):
    # compute the sd_param and hessian matrix
    sd_param_matrix = np.dot(steep_descent.T, error.T)
    hessian_matrix = np.dot(steep_descent.T, steep_descent)
    hessian_matrix_inv = np.linalg.pinv(hessian_matrix)
    
    # use the above two matrices to get the error in p matrix and return
    delta_p = np.dot(hessian_matrix_inv, sd_param_matrix)
    return delta_p    
    
# compute the steep descent using two images and the coordinates of the ROI of two images
def get_steep_descent(sobelx, sobely, template_gray_image_coordinates_array, new_gray_image_coordinates_array):
    
    # get the pixel array for sobelx
    sobelx_pixel_array = get_pixel_array(sobelx, new_gray_image_coordinates_array)
    
    # get the pixel array for sobely
    sobely_pixel_array = get_pixel_array(sobely, new_gray_image_coordinates_array)
    
    # get four images
    image1 = sobelx_pixel_array * template_gray_image_coordinates_array[0, :]
    image2 = sobely_pixel_array * template_gray_image_coordinates_array[0, :]
    image3 = sobelx_pixel_array * template_gray_image_coordinates_array[1, :]
    image4 = sobely_pixel_array * template_gray_image_coordinates_array[1, :]
    
    # return the six images
    return np.vstack((image1, image2, image3, image4, sobelx_pixel_array, sobely_pixel_array)).T
    
# lucas kanade algorithm
def lucas_kanade_algorithm(template_gray_image, current_gray_image, sobelx, sobely, x_range, y_range, thresh):
    
    # get the coordinates of the ROI for template image
    template_gray_image_coordinates_array = get_coordinates_array(x_range, y_range)
    
    # define p matrix, used for calculating the warped image
    p = np.array([[0, 0, 0, 0, 0, 0]]).T
    
    # get the coordinates of the ROI in the new frame
    (new_template_image_coordinates_array, new_rectangle_coordinates) = get_new_image_coordinates(template_gray_image_coordinates_array, p, x_range, y_range)
    
    # get the pixel array of the template image
    template_pixel_array = get_pixel_array(template_gray_image, new_template_image_coordinates_array)
    
    # run algorithm
    while(True):
        
        # get the coordinates of the ROI in the new frame
        (new_gray_image_coordinates_array, new_rectangle_coordinates) = get_new_image_coordinates(template_gray_image_coordinates_array, p, x_range, y_range)
        
        # if new coordinates not in range, then break
        if(new_gray_image_coordinates_array[0, 0] < 0 or new_gray_image_coordinates_array[1, 0] < 0 or new_gray_image_coordinates_array[0, new_gray_image_coordinates_array.shape[1] - 1] > current_gray_image.shape[1] or new_gray_image_coordinates_array[1, new_gray_image_coordinates_array.shape[1] - 1] > current_gray_image.shape[0]):
            break
            
        # get the pixel array of the gray image
        new_pixel_array = get_pixel_array(current_gray_image, new_gray_image_coordinates_array)
        
        # compute the error
        error = compute_error_images(template_pixel_array, new_pixel_array)
        
        # compute steep descent
        steep_descent = get_steep_descent(sobelx, sobely, template_gray_image_coordinates_array, new_gray_image_coordinates_array)
        
        # get the delta_p
        delta_p = get_delta_p(error, steep_descent)
        
        # get p norm and update p
        p_norm = np.linalg.norm(delta_p)
        p = np.reshape(p, (6, 1))
        p = p + delta_p
        
        # if p_norm within thresh break
        if(p_norm < thresh):
            break
            
    # return the ROI needed for this frame
    return new_rectangle_coordinates

In [13]:
threshold = 0.01
x_range = np.array([269, 303])
y_range = np.array([75, 139])
template_image = cv2.imread(files[0])
gray_template_image = get_grayscale_image(template_image)

In [16]:
# read files
for file in files:
    # read image
    image = cv2.imread(file)
    image_copy = image.copy()
    
    # get grayscale image
    gray = get_grayscale_image(image_copy)
    gray = update_grayscale_image(gray_template_image, gray)
    
    # compute derivatives around x and y directions
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = 3)
    sobelx = cv2.convertScaleAbs(sobelx)
    sobely = cv2.convertScaleAbs(sobely)
    cv2.imshow("Sobelx", sobelx)
    cv2.waitKey(0)
    
    
    image = cv2.rectangle(image, (269, 75), (303, 139), (0, 0, 255), 2)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    break